In [ ]:
from rich import print as rprint

import arff



import numpy as np
import pandas as pd



import matplotlib.pyplot as plt


from IPython.core.interactiveshell import InteractiveShell


InteractiveShell.ast_node_interactivity = "all"
pd.set_option("display.precision", 2)


### Explorative Datenanalyse

Der zweite Vertrag hat sechs Verträge mehr als der erste Vertrag.


In [ ]:
df = pd.read_csv("ins_claims.csv", index_col=0)

rprint(df.info())

df.describe(include=["float64", "object"])


In [ ]:
up_lim = df["ClaimAmount"].quantile(0.995)
df = df[df["ClaimAmount"] < up_lim]

for key_ in df.columns:
    df.plot.scatter(x="IDpol", y=key_)


In [14]:
df_sev = df[df["ClaimAmount"] > 0]
df_sev.corr(numeric_only=True)


,IDpol,ClaimAmount,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Density
IDpol,1.00,-2.22e-01,4.43e-02,-1.02e-01,4.35e-02,-0.06,0.02,-0.04,0.06
ClaimAmount,-0.22,1.00e+00,6.49e-03,2.11e-02,-2.33e-02,0.03,0.06,0.12,-0.03
ClaimNb,0.04,6.49e-03,1.00e+00,-4.32e-03,-4.72e-03,-0.02,-0.02,0.02,0.03
Exposure,-0.10,2.11e-02,-4.32e-03,1.00e+00,-1.98e-02,-0.03,0.20,-0.16,-0.08
VehPower,0.04,-2.33e-02,-4.72e-03,-1.98e-02,1.00e+00,-0.01,0.02,-0.06,0.02
VehAge,-0.06,3.11e-02,-1.51e-02,-2.84e-02,-1.34e-02,1.00,-0.07,0.13,-0.06
DrivAge,0.02,6.11e-02,-1.90e-02,1.97e-01,2.10e-02,-0.07,1.00,-0.45,-0.03
BonusMalus,-0.04,1.16e-01,1.57e-02,-1.64e-01,-6.25e-02,0.13,-0.45,1.00,0.09
Density,0.06,-3.29e-02,3.43e-02,-7.70e-02,1.68e-02,-0.06,-0.03,0.09,1.00


In [ ]:
rprint(sum(df["ClaimAmount"] > 0.0) / df.shape[0])
rprint(sum(df["ClaimNb"] > 0.0) / df.shape[0])


## Feature Engineering


## Modellvergleich


## Modellbuilding
